In [1]:
### ValDXer testing
import os
os.environ['HDXER_PATH'] = '/home/alexi/Documents/HDXer'


In [2]:

from ValDX.ValidationDX import ValDXer
from ValDX.VDX_Settings import Settings

settings = Settings(name='test')
settings.replicates = 5
settings.gamma_range = (2,10)
settings.train_frac = 0.8
settings.times = [0.0, 0.5, 5.0]
import pickle

VDX = ValDXer(settings)
expt_name = 'HOIP_dab3'
test_name = "HOIPdab3"


/home/alexi/Documents/ValDX


/home/alexi/miniconda3/envs/HDXER_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to 'HDXER_ENV' environment: /home/alexi/miniconda3/envs/HDXER_ENV
Experiment name test already exists. Attempting to change name to test0
Experiment name test already exists. Attempting to change name to test1
Experiment name test already exists. Attempting to change name to test2
Experiment name test already exists. Attempting to change name to test3
Experiment name test already exists. Attempting to change name to test4
Experiment name test already exists. Attempting to change name to test5
Experiment name test already exists. Attempting to change name to test6
Experiment name test already exists. Attempting to change name to test7
Experiment name test already exists. Attempting to change name to test8
Experiment name test already exists. Attempting to change name to test9


In [3]:
import os
print(os.getenv('HDXER_PATH'))


/home/alexi/Documents/HDXer


In [4]:
print(os.environ["HDXER_PATH"])
print(__name__)

/home/alexi/Documents/HDXer
__main__


In [5]:
import pandas as pd
raw_csv = os.path.join("raw_data", "HOIP","dab3","dab3_3_excel.csv")

raw_df = pd.read_csv(raw_csv)
# remove multilevel columns


In [6]:
# print(raw_df.head())

# select state dab3
dab3_df = raw_df[raw_df['State'] == 'dAb3_3']


# add 697 to Start and End  
dab3_df['Start'] = dab3_df['Start'] + 697
dab3_df['End'] = dab3_df['End'] + 697

# add UptakeFraction column
dab3_df['UptakeFraction'] = dab3_df['Uptake'] / dab3_df['MaxUptake']
dab3_df.tail()


,Protein,Unnamed: 1,Unnamed: 2,Start,End,Sequence,Modification,Fragment,MaxUptake,MHP,State,Exposure,Center,Center SD,Uptake,Uptake SD,RT,RT SD,UptakeFraction
322,HOIP-RBR,1032,1056,1033,1057,ATERYLHVRPQPLAGEDPPAYQARL,NaN,NaN,20,2848.4853,dAb3_3,5.0,2855.389752,0.050030,5.176299,0.054997,6.028652,0.002530,0.258815
323,HOIP-RBR,1034,1056,1035,1057,ERYLHVRPQPLAGEDPPAYQARL,NaN,NaN,18,2676.4005,dAb3_3,5.0,2682.862836,0.078566,4.863580,0.088043,6.013863,0.002566,0.270199
324,HOIP-RBR,1035,1056,1036,1057,RYLHVRPQPLAGEDPPAYQARL,NaN,NaN,17,2547.3579,dAb3_3,5.0,2553.745613,0.049922,4.922553,0.086997,5.958447,0.002980,0.289562
325,HOIP-RBR,1038,1056,1039,1057,HVRPQPLAGEDPPAYQARL,NaN,NaN,14,2115.1094,dAb3_3,5.0,2120.299467,0.065218,3.958915,0.107950,5.716731,0.001727,0.282780
326,HOIP-RBR,1057,1075,1058,1076,LQKLTEEVPLGQSIPRRRK,NaN,NaN,16,2248.3248,dAb3_3,5.0,2255.862180,0.101628,6.266518,0.104689,5.058567,0.003351,0.391657


In [7]:
columns_to_drop = ["Protein", "Sequence", "Fragment", "Modification", "State", "MaxUptake", "Uptake", "MHP", "Center", "Center SD", "Uptake", "Uptake SD", "RT", "RT SD"]
dab3_df = dab3_df.drop(columns_to_drop, axis=1)
dab3_df.head()

,Unnamed: 1,Unnamed: 2,Start,End,Exposure,UptakeFraction
0,697,703,698,704,0.0,0.0
1,702,714,703,715,0.0,0.0
2,702,717,703,718,0.0,0.0
3,702,721,703,722,0.0,0.0
4,703,714,704,715,0.0,0.0


In [8]:
dab3_df= dab3_df.drop(columns=['Unnamed: 1','Unnamed: 2'])
dab3_df.head()


,Start,End,Exposure,UptakeFraction
0,698,704,0.0,0.0
1,703,715,0.0,0.0
2,703,718,0.0,0.0
3,703,722,0.0,0.0
4,704,715,0.0,0.0


In [9]:

# pivot exposure and uptake fraction
dab3_df = dab3_df.groupby(['Start', 'End', 'Exposure'])['UptakeFraction'].mean().reset_index()

dab3_df.head()


,Start,End,Exposure,UptakeFraction
0,698,704,0.0,0.000000
1,698,704,0.5,0.466735
2,698,704,5.0,0.435965
3,703,715,0.0,0.000000
4,703,715,0.5,0.329021


In [10]:
dab3_df.columns

Index(['Start', 'End', 'Exposure', 'UptakeFraction'], dtype='object')

In [11]:

# remove End > 1072
dab3_df = dab3_df.loc[dab3_df['End'] <= 1072]
dab3_df.head()


,Start,End,Exposure,UptakeFraction
0,698,704,0.0,0.000000
1,698,704,0.5,0.466735
2,698,704,5.0,0.435965
3,703,715,0.0,0.000000
4,703,715,0.5,0.329021


In [12]:
dab3_df = dab3_df.pivot(index=['Start','End'], columns='Exposure', values='UptakeFraction')


In [13]:

# print entire dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(dab3_df)


Exposure    0.0       0.5       5.0
Start End                          
698   704   0.0  0.466735  0.435965
703   715   0.0  0.329021  0.417331
      718   0.0  0.437678  0.514648
      722   0.0  0.494072  0.550188
704   715   0.0  0.409742  0.499738
      718   0.0  0.465488  0.532689
      722   0.0  0.518648  0.575634
705   715   0.0  0.428948  0.505906
      717   0.0  0.487081  0.544802
      718   0.0  0.490184  0.554119
      721   0.0  0.522236  0.584780
708   715   0.0  0.490079  0.531893
      717   0.0  0.543597  0.572614
      718   0.0  0.548075  0.583051
      720   0.0  0.603121  0.626074
      722   0.0  0.575935  0.598220
716   722   0.0  0.560898  0.587244
723   729   0.0  0.162769  0.272038
      730   0.0  0.163224  0.282758
724   730   0.0  0.159595  0.267070
730   737   0.0  0.309840  0.407468
731   737   0.0  0.325881  0.412225
      738   0.0  0.317411  0.423762
738   745   0.0  0.381343  0.389136
      746   0.0  0.356548  0.373868
      748   0.0  0.351628  0

In [14]:
#print row 3
print(dab3_df.iloc[0])


Exposure
0.0    0.000000
0.5    0.466735
5.0    0.435965
Name: (698, 704), dtype: float64


In [15]:
# fill in missing

dab3_df.reset_index(inplace=True)

# Forward fill the 'Start' column
dab3_df['Start'] = dab3_df['Start'].ffill()

# Set 'Start' and 'End' back as the index if needed
dab3_df.set_index(['Start', 'End'], inplace=True)


In [16]:
# save with space delimiter
# round to 5 dp

dab3_df = dab3_df.round(5)
dab3_df.to_csv(os.path.join("raw_data", "HOIP", 'HOIP_dab3_dfs.csv'), sep=' ')

In [17]:




dab3_df.to_csv(os.path.join("raw_data", "HOIP", "dab3.csv"), index=False)

In [35]:
# select only Start and End
dab3_df.reset_index(inplace=True)

dab3_segs = dab3_df[['Start', 'End']]
dab3_segs.drop(continu)

dab3_segs.head()

Exposure,Start,End
0,698,704
1,703,715
2,703,718
3,703,722
4,704,715


In [18]:
import subprocess
from ValDX.helpful_funcs import conda_to_env_dict

# Assuming settings.HDXer_env contains the name of your Conda environment
env_path = conda_to_env_dict(settings.HDXer_env)

command = "echo $HDXER_PATH"
print("command:", command)

# Run the command in the subprocess
output = subprocess.run(command, shell=True, env=env_path, capture_output=True, text=True)

# Capture and print the standard output (stdout)
hdxer_path = output.stdout.strip()  # .strip() removes any trailing newline
print("HDXER_PATH:", hdxer_path)


Path to 'HDXER_ENV' environment: /home/alexi/miniconda3/envs/HDXER_ENV
command: echo $HDXER_PATH
HDXER_PATH: /home/alexi/Documents/HDXer


In [19]:
# BPTI data
BPTI_dir = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI"
BPTI_dir = "/home/alexi/Documents/ValDX/raw_data/HDXer_tutorial/BPTI"



In [20]:
expt_dir = "/home/alexi/Documents/ValDX/raw_data/HOIP/dab3"

os.listdir(expt_dir)

segs_name = "HOIP_dab3_segs.txt"
segs_path = os.path.join(expt_dir, segs_name)

hdx_name = "HOIP_dab3_dfs.dat"
hdx_path = os.path.join(expt_dir, hdx_name)
print(hdx_path)

rates_name = "out__train_HOIPdab3_1Intrinsic_rates.dat" #need to correct this
rates_path =os.path.join(expt_dir, rates_name)


/home/alexi/Documents/ValDX/raw_data/HOIP/dab3/HOIP_dab3_dfs.dat


In [21]:

VDX.load_HDX_data(HDX_path=hdx_path, SEG_path=segs_path, calc_name=expt_name)
VDX.load_intrinsic_rates(path=rates_path, calc_name=expt_name)


Preparing HDX data for HOIP_dab3
Path /home/alexi/Documents/ValDX/raw_data/HOIP/dab3/HOIP_dab3_dfs.dat
AVG: ncol = 5, len(names) = 3


In [22]:
print(VDX.rates)

Empty DataFrame
Columns: []
Index: []


In [23]:
print(VDX.paths)


                                                 HDX  \
0  /home/alexi/Documents/ValDX/raw_data/HOIP/dab3...   
1                                                NaN   

                                                 SEG  calc_name  experimental  \
0  /home/alexi/Documents/ValDX/raw_data/HOIP/dab3...  HOIP_dab3         False   
1                                                NaN  HOIP_dab3         False   

                                           int_rates  
0                                                NaN  
1  /home/alexi/Documents/ValDX/raw_data/HOIP/dab3...  


In [24]:
HOIP_dir = "/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3"
# list of folders in HOIP_dir
import glob
reps = 2


reps_list = [os.path.join(HOIP_dir, f"R_{i}") for i in range(1, reps+1)]

print(reps_list)


traj_names = ["APO_dab3_6SC6_7-nojump.xtc"]

#


['/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_1', '/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_2']


In [25]:
sim_name = "HOIP_dab3_APO"
top_name = "APO_dab3_6SC6_7-nojump.pdb"
top_path = os.path.join(HOIP_dir, top_name)
# reps =100

#traj_paths is a list of every combination of rep_paths and traj_names

traj_paths = []
for rep_path in reps_list:
    for traj_name in traj_names:
        path = os.path.join(rep_path, traj_name)
        print(path)
        traj_paths.append(path)

print(top_path)


/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_1/APO_dab3_6SC6_7-nojump.xtc
/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_2/APO_dab3_6SC6_7-nojump.xtc
/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/APO_dab3_6SC6_7-nojump.pdb


In [26]:
print(traj_paths)
VDX.load_structures(top_path=top_path, traj_paths=traj_paths, calc_name=test_name)

['/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_1/APO_dab3_6SC6_7-nojump.xtc', '/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_2/APO_dab3_6SC6_7-nojump.xtc']
                                                 HDX  \
0  /home/alexi/Documents/ValDX/raw_data/HOIP/dab3...   
1                                                NaN   
2                                                NaN   

                                                 SEG  calc_name experimental  \
0  /home/alexi/Documents/ValDX/raw_data/HOIP/dab3...  HOIP_dab3        False   
1                                                NaN  HOIP_dab3        False   
2                                                NaN   HOIPdab3          NaN   

                                           int_rates  \
0                                                NaN   
1  /home/alexi/Documents/ValDX/raw_data/HOIP/dab3...   
2                                                NaN   

                                                 top  

In [27]:
# import MDAnalysis as mda
# from MDAnalysis.coordinates.DCD import DCDWriter  # Import DCDWriter directly

# small_traj_paths = []

# for rep in range(len(traj_paths)):
#     traj = traj_paths[rep]
#     u = mda.Universe(top_path, traj)
#     print(u.trajectory.n_frames)

#     # sample by stride settings.stide

#     small_traj_name = traj.replace(".dcd", f"_str{settings.stride}.dcd")

#     # Create an AtomGroup from the Universe's selection
#     ag = u.select_atoms('all')

#     # write traj
#     with DCDWriter(small_traj_name, ag.n_atoms) as writer:
#         for ts in u.trajectory[::settings.stride]:
#             writer.write(ag)

#     small_traj_paths.append(small_traj_name)

In [28]:
print((traj_paths))

['/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_1/APO_dab3_6SC6_7-nojump.xtc', '/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/R_2/APO_dab3_6SC6_7-nojump.xtc']


In [29]:
# VDX.load_structures(top_path=top_path, traj_paths=traj_paths, calc_name=test_name)


In [30]:
VDX.paths.top.dropna().values[0]

'/home/alexi/Documents/xMD-HOIP/data/MD/6SC6/APO_dab3/APO_dab3_6SC6_7-nojump.pdb'

In [31]:
train_dfs, val_dfs, train_gammas, val_gammas = VDX.run_VDX(calc_name=test_name, expt_name=expt_name)

Random seeds: [42, 43, 44, 45, 46]
Randomly splitting segments for HOIPdab3 with random seed 42 and train fraction 0.8
Saved train HOIPdab3 segments to /home/alexi/Documents/ValDX/data/test9/train_HOIPdab3_1/train_residue_segs__HOIPdab3_.txt
Train Peptide numbers: [  0   2   3   4   5   6   7   8   9  10  11  12  14  15  16  17  18  19
  22  23  24  25  26  27  28  29  30  31  32  33  35  36  37  38  39  40
  41  42  44  45  46  47  51  55  56  57  59  60  61  62  63  64  65  66
  67  68  69  70  72  73  74  75  76  77  78  79  81  82  83  84  85  86
  89  90  92  93  94  95  96  97  98 100 101 103 104 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 122 123 124 125 126 127 128 129 131
 132 133 135 136 137 138 139 140 141 142 143 144 145 146 147 148 150 152
 153 154 155 156 157 158 159 162 163 164 165 167 168 170 171 172 173 175
 176 177 178 179 180 181 182 183 184 185 186 192 193 194 195 196 197 198
 199 200 202 203 204 206 208 209 210 211 213 215 216 218 219 220 221 222
 223 

IndexError: positional indexers are out-of-bounds

In [32]:
VDX.HDX_data

,0.0,0.5,5.0,peptide,calc_name
0,0.0,0.46673,0.43596,0,HOIP_dab3
1,0.0,0.32902,0.41733,1,HOIP_dab3
2,0.0,0.43768,0.51465,2,HOIP_dab3
3,0.0,0.49407,0.55019,3,HOIP_dab3
4,0.0,0.40974,0.49974,4,HOIP_dab3
5,0.0,0.46549,0.53269,5,HOIP_dab3
6,0.0,0.51865,0.57563,6,HOIP_dab3
7,0.0,0.42895,0.50591,7,HOIP_dab3
8,0.0,0.48708,0.54480,8,HOIP_dab3
9,0.0,0.49018,0.55412,9,HOIP_dab3


In [ ]:
VDX.save_experiment()

dump = train_dfs, val_dfs, train_gammas, val_gammas

import pickle
with open(f'dump_{VDX.name}.pickle', 'wb') as f:
    print(f"Saving pickle", f'dump_{VDX.name}.pickle')
    pickle.dump(dump, f)

In [ ]:
break

In [ ]:
# VDX = VDX.load_experiment(load_path='logs/test_full0.50_1705421531.pkl')

In [ ]:
with open('dump_test_full0.50.pickle', 'rb') as f:
    dump = pickle.load(f)

train_dfs, val_dfs, train_gammas, val_gammas = dump

VDX.evaluate_HDX(train_dfs=train_dfs, 
                 val_dfs=val_dfs, 
                 calc_name=test_name,
                 expt_name=expt_name,
                 train_gammas=train_gammas, 
                 val_gammas=val_gammas, 
                 n_reps=VDX.settings.replicates,
                   )

In [ ]:
import pandas as pd

In [ ]:
# file_name = "covid_submissions_all_info.csv"

In [ ]:
# sub_test = pd.read_csv(file_name)

In [ ]:
# sub_test.columns.to_list()